In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#Reading the Receipts dataset
rec_df = pd.read_json('receipts.json.gz',lines=True,compression='gzip')

In [3]:
rec_df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


### In the receipts datasets, we notice that the columns have value in the dictionary format. In order to clean the data the first step would be to transform the columns into a readable pandas dataframe to perform operations on.

In [4]:
rec_df['_id'] = rec_df['_id'].apply(lambda x: x['$oid'])

In [5]:
rec_df.rename(columns = {'_id' : 'receipt_id'}, inplace = True)

In [6]:
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   receipt_id               1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [7]:
from datetime import datetime

In [8]:
rec_df['createDate'] = rec_df['createDate'].apply(lambda x: x['$date'])
rec_df['dateScanned'] = rec_df['dateScanned'].apply(lambda x: x['$date'])
rec_df['modifyDate'] = rec_df['modifyDate'].apply(lambda x: x['$date'])

In [9]:
rec_df['createDate'] = rec_df['createDate'].apply(lambda x : pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))
rec_df['dateScanned'] = rec_df['dateScanned'].apply(lambda x : pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))
rec_df['modifyDate'] = rec_df['modifyDate'].apply(lambda x : pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))

### Majority of columns in the dataset consists of null-values, thus we cannot directly apply the date transformations in `finished_date, pointsAwardedDate and purchaseDate`.

In [10]:
rec_df['finishedDate'].fillna('None', inplace = True)
rec_df['pointsAwardedDate'].fillna('None', inplace = True)
rec_df['purchaseDate'].fillna('None', inplace = True)

In [11]:
rec_df['finishedDate'] = rec_df['finishedDate'].apply(lambda x: 'None' if x == 'None' else int(x['$date']))
rec_df['pointsAwardedDate'] = rec_df['pointsAwardedDate'].apply(lambda x: 'None' if x == 'None' else int(x['$date']))
rec_df['purchaseDate'] = rec_df['purchaseDate'].apply(lambda x: 'None' if x == 'None' else int(x['$date']))


In [12]:
rec_df['finishedDate'] = rec_df['finishedDate'].apply(lambda x : 'None' if x == 'None' else pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))

rec_df['pointsAwardedDate']= rec_df['pointsAwardedDate'].apply(lambda x : 'None' if x == 'None' else pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))
rec_df['purchaseDate'] = rec_df['purchaseDate'].apply(lambda x : 'None' if x == 'None' else pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))


In [13]:
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   receipt_id               1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             1119 non-null   object        
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        1119 non-null   object        
 8   pointsEarned             609 non-null    float64       
 9   purchaseDate             1119 non-null   object        
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptItemList   679 non-null    object        
 12  rewardsReceiptStatus     1119 non-

## Diving deep into specific columns

In [14]:
rec_df['rewardsReceiptItemList']

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
2       [{'needsFetchReview': False, 'partnerItemId': ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
1114    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1115                                                  NaN
1116                                                  NaN
1117    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 1119, dtype: object

### We see that `rewardsReceiptItemList` column has many key value pairs where keys are additional column names and values are the respective key column values.

In [15]:
rec_df['rewardsReceiptItemList'][2][0]

{'needsFetchReview': False,
 'partnerItemId': '1',
 'preventTargetGapPoints': True,
 'userFlaggedBarcode': '4011',
 'userFlaggedNewItem': True,
 'userFlaggedPrice': '26.00',
 'userFlaggedQuantity': 3}

In [16]:
rec_df['rewardsReceiptItemList'][1][0]


{'barcode': '4011',
 'description': 'ITEM NOT FOUND',
 'finalPrice': '1',
 'itemPrice': '1',
 'partnerItemId': '1',
 'quantityPurchased': 1}

In [17]:
rec_df['rewardsReceiptItemList'][0][0]

{'barcode': '4011',
 'description': 'ITEM NOT FOUND',
 'finalPrice': '26.00',
 'itemPrice': '26.00',
 'needsFetchReview': False,
 'partnerItemId': '1',
 'preventTargetGapPoints': True,
 'quantityPurchased': 5,
 'userFlaggedBarcode': '4011',
 'userFlaggedNewItem': True,
 'userFlaggedPrice': '26.00',
 'userFlaggedQuantity': 5}

In [18]:
rec_df['rewardsReceiptItemList'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1119 entries, 0 to 1118
Series name: rewardsReceiptItemList
Non-Null Count  Dtype 
--------------  ----- 
679 non-null    object
dtypes: object(1)
memory usage: 8.9+ KB


### When we see into the values of rewardsReceiptItemList column, we see that there are diiferent entries for different item report  and there are many null values in the column as well.

### Thus to normalize it, we have to first take the non-null values and normalize those values. After that, join back to receipts df column.

In [19]:
rec_df_non_null = rec_df[rec_df['rewardsReceiptItemList'].notnull()]

In [20]:
rec_df_non_null.reset_index(inplace=True)

In [21]:
rec_df_non_null.shape

(679, 16)

## We know that the values in `rewardsReceiptItemList` columns are in list, so first to extract only the dictionary part to normalize it.

In [22]:
rec_df_non_null['rewardsReceiptItemList'] = rec_df_non_null['rewardsReceiptItemList'].apply(lambda x: x[0])

/var/folders/dl/zfkvqms549vchyzbsj30cgbw0000gn/T/ipykernel_88666/1675703242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_df_non_null['rewardsReceiptItemList'] = rec_df_non_null['rewardsReceiptItemList'].apply(lambda x: x[0])


In [23]:
rec_df_non_null.shape

(679, 16)

In [24]:
rewards_item_list = pd.json_normalize(rec_df_non_null['rewardsReceiptItemList'])
rewards_item_list = rewards_item_list.add_prefix('rrIL.')

In [25]:
rewards_item_list.shape

(679, 32)

In [26]:
rec_df_non_null = pd.concat([rec_df_non_null,rewards_item_list], axis =1)

In [27]:
# rec_df_non_null = pd.merge(rec_df_non_null, rewards_item_list, left_index=True, right_index=True, how='outer')

In [28]:
rec_df_non_null.shape

(679, 48)

In [29]:
pd.set_option('display.max_columns',None)

In [30]:
rec_df_non_null.drop('index',inplace=True, axis=1)

In [31]:
rec_df_non_null.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,rrIL.barcode,rrIL.description,rrIL.finalPrice,rrIL.itemPrice,rrIL.needsFetchReview,rrIL.partnerItemId,rrIL.preventTargetGapPoints,rrIL.quantityPurchased,rrIL.userFlaggedBarcode,rrIL.userFlaggedNewItem,rrIL.userFlaggedPrice,rrIL.userFlaggedQuantity,rrIL.originalMetaBriteBarcode,rrIL.originalMetaBriteDescription,rrIL.pointsNotAwardedReason,rrIL.pointsPayerId,rrIL.rewardsGroup,rrIL.rewardsProductPartnerId,rrIL.brandCode,rrIL.competitorRewardsGroup,rrIL.discountedItemPrice,rrIL.originalReceiptItemText,rrIL.itemNumber,rrIL.needsFetchReviewReason,rrIL.originalMetaBriteQuantityPurchased,rrIL.pointsEarned,rrIL.targetPrice,rrIL.competitiveProduct,rrIL.userFlaggedDescription,rrIL.deleted,rrIL.priceAfterCoupon,rrIL.metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,None,2021-01-03 10:25:42,None,5.0,2021-01-02 19:00:00,1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
rec_df.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,None,2021-01-03 10:25:42,None,5.0,2021-01-02 19:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [33]:
rec_df.columns

Index(['receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId'],
      dtype='object')

In [34]:
rec_df_non_null.columns

Index(['receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', 'rrIL.barcode', 'rrIL.description',
       'rrIL.finalPrice', 'rrIL.itemPrice', 'rrIL.needsFetchReview',
       'rrIL.partnerItemId', 'rrIL.preventTargetGapPoints',
       'rrIL.quantityPurchased', 'rrIL.userFlaggedBarcode',
       'rrIL.userFlaggedNewItem', 'rrIL.userFlaggedPrice',
       'rrIL.userFlaggedQuantity', 'rrIL.originalMetaBriteBarcode',
       'rrIL.originalMetaBriteDescription', 'rrIL.pointsNotAwardedReason',
       'rrIL.pointsPayerId', 'rrIL.rewardsGroup',
       'rrIL.rewardsProductPartnerId', 'rrIL.brandCode',
       'rrIL.competitorRewardsGroup', 'rrIL.discountedItemPrice',
       'rrIL.originalReceiptItemText', 'rrIL.itemNumber',
       'rrIL.needsFetchReviewReason',

### Concatinating back to original dataframe

In [35]:
rec_df = pd.merge(rec_df,rec_df_non_null[['receipt_id','userId','rrIL.barcode','rrIL.description', 'rrIL.finalPrice',
       'rrIL.itemPrice', 'rrIL.needsFetchReview', 'rrIL.partnerItemId',
       'rrIL.preventTargetGapPoints', 'rrIL.quantityPurchased',
       'rrIL.userFlaggedBarcode', 'rrIL.userFlaggedNewItem',
       'rrIL.userFlaggedPrice', 'rrIL.userFlaggedQuantity',
       'rrIL.originalMetaBriteBarcode', 'rrIL.originalMetaBriteDescription',
       'rrIL.pointsNotAwardedReason', 'rrIL.pointsPayerId',
       'rrIL.rewardsGroup', 'rrIL.rewardsProductPartnerId', 'rrIL.brandCode',
       'rrIL.competitorRewardsGroup', 'rrIL.discountedItemPrice',
       'rrIL.originalReceiptItemText', 'rrIL.itemNumber',
       'rrIL.needsFetchReviewReason',
       'rrIL.originalMetaBriteQuantityPurchased', 'rrIL.pointsEarned',
       'rrIL.targetPrice', 'rrIL.competitiveProduct',
       'rrIL.userFlaggedDescription', 'rrIL.deleted', 'rrIL.priceAfterCoupon',
       'rrIL.metabriteCampaignId']], left_on=['receipt_id','userId'],right_on=['receipt_id','userId'], how='left')

In [36]:
rec_df.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,rrIL.barcode,rrIL.description,rrIL.finalPrice,rrIL.itemPrice,rrIL.needsFetchReview,rrIL.partnerItemId,rrIL.preventTargetGapPoints,rrIL.quantityPurchased,rrIL.userFlaggedBarcode,rrIL.userFlaggedNewItem,rrIL.userFlaggedPrice,rrIL.userFlaggedQuantity,rrIL.originalMetaBriteBarcode,rrIL.originalMetaBriteDescription,rrIL.pointsNotAwardedReason,rrIL.pointsPayerId,rrIL.rewardsGroup,rrIL.rewardsProductPartnerId,rrIL.brandCode,rrIL.competitorRewardsGroup,rrIL.discountedItemPrice,rrIL.originalReceiptItemText,rrIL.itemNumber,rrIL.needsFetchReviewReason,rrIL.originalMetaBriteQuantityPurchased,rrIL.pointsEarned,rrIL.targetPrice,rrIL.competitiveProduct,rrIL.userFlaggedDescription,rrIL.deleted,rrIL.priceAfterCoupon,rrIL.metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,None,2021-01-03 10:25:42,None,5.0,2021-01-02 19:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
rec_df.shape

(1119, 47)

In [38]:
rec_df.columns

Index(['receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', 'rrIL.barcode', 'rrIL.description',
       'rrIL.finalPrice', 'rrIL.itemPrice', 'rrIL.needsFetchReview',
       'rrIL.partnerItemId', 'rrIL.preventTargetGapPoints',
       'rrIL.quantityPurchased', 'rrIL.userFlaggedBarcode',
       'rrIL.userFlaggedNewItem', 'rrIL.userFlaggedPrice',
       'rrIL.userFlaggedQuantity', 'rrIL.originalMetaBriteBarcode',
       'rrIL.originalMetaBriteDescription', 'rrIL.pointsNotAwardedReason',
       'rrIL.pointsPayerId', 'rrIL.rewardsGroup',
       'rrIL.rewardsProductPartnerId', 'rrIL.brandCode',
       'rrIL.competitorRewardsGroup', 'rrIL.discountedItemPrice',
       'rrIL.originalReceiptItemText', 'rrIL.itemNumber',
       'rrIL.needsFetchReviewReason',

In [39]:
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1119 entries, 0 to 1118
Data columns (total 47 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   receipt_id                               1119 non-null   object        
 1   bonusPointsEarned                        544 non-null    float64       
 2   bonusPointsEarnedReason                  544 non-null    object        
 3   createDate                               1119 non-null   datetime64[ns]
 4   dateScanned                              1119 non-null   datetime64[ns]
 5   finishedDate                             1119 non-null   object        
 6   modifyDate                               1119 non-null   datetime64[ns]
 7   pointsAwardedDate                        1119 non-null   object        
 8   pointsEarned                             609 non-null    float64       
 9   purchaseDate                             

In [40]:
rec_df.dateScanned.max()

Timestamp('2021-03-01 18:17:34')

In [41]:
recent_month = rec_df[(rec_df['dateScanned']>='2021-02-01 00:00:00') & (rec_df['dateScanned']<='2021-03-01 00:00:00')]

In [42]:
recent_month.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448 entries, 639 to 1118
Data columns (total 47 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   receipt_id                               448 non-null    object        
 1   bonusPointsEarned                        87 non-null     float64       
 2   bonusPointsEarnedReason                  87 non-null     object        
 3   createDate                               448 non-null    datetime64[ns]
 4   dateScanned                              448 non-null    datetime64[ns]
 5   finishedDate                             448 non-null    object        
 6   modifyDate                               448 non-null    datetime64[ns]
 7   pointsAwardedDate                        448 non-null    object        
 8   pointsEarned                             88 non-null     float64       
 9   purchaseDate                            

In [43]:
rec_df.to_csv('receipts_final.csv')